# Crea estructura de datos pares de relays


In [3]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

"""
Genera independent_relay_pairs_transformer.json
leyendo los ajustes TDS / pickup de
TDS_Pickup_todos_escenarios.csv, respetando
el orden original de los renglones del CSV.
"""

import json, os, copy, csv, numpy as np
from collections import defaultdict, deque

# ──────────────────────────── CONSTANTES ─────────────────────────────
K, N, DEC = 0.14, 0.02, 4

# ──────────────────────────── RUTAS ──────────────────────────────────
BASE = "/Users/gustavo/Documents/Projects/TESIS_UNAL/ADAPTIVE_ALGORITHM/"

DATA_COORD_FILE  = BASE + "data/raw/data_coordination.json"
CSV_VALUES_FILE  = BASE + "data/processed/model/results/TDS_Pickup_todos_escenarios.csv"
OUTPUT_PAIRS_FILE = BASE + "data/processed/independent_relay_pairs_best.json"

# ──────────────────────────── AUXILIARES ─────────────────────────────
def eliminar_timestamp(obj):
    if isinstance(obj, dict):
        obj.pop("timestamp", None)
        for v in obj.values(): eliminar_timestamp(v)
    elif isinstance(obj, list):
        for e in obj: eliminar_timestamp(e)

def calc_timeout(Ishc, Ipi, TDS):
    if not all(isinstance(x, (int, float)) for x in [Ishc, Ipi, TDS]):      return 0.0
    if any(x is None or x <= 0 for x in [Ipi, TDS]) or Ishc <= Ipi:          return 0.0
    try:
        denom = (Ishc / Ipi) ** N - 1
        if abs(denom) < 1e-9: return 0.0
        t = (K / denom) * TDS
        return round(t, DEC) if np.isfinite(t) else 0.0
    except Exception:
        return 0.0

# ───────────────────── LEER CSV (en orden) ───────────────────────────
print(f"▶ Leyendo ajustes de: {CSV_VALUES_FILE}")
csv_queues = defaultdict(deque)      # {escenario: deque([row1,row2,...])}

with open(CSV_VALUES_FILE, newline="") as f:
    rdr = csv.DictReader(f)
    for row in rdr:
        esc = row["escenario"]                    # encabezado exacto
        csv_queues[esc].append({
            "TDS_principal":  float(row["TDS_principal"]),
            "Pickup_princ":   float(row["Pickup_principal"]),
            "TDS_respaldo":   float(row["TDS_respaldo"]),
            "Pickup_resp":    float(row["Pickup_respaldo"]),
        })

print(f"   Escenarios encontrados en CSV: {list(csv_queues.keys())}")

# ───────────────────── LEER JSON BASE ────────────────────────────────
print(f"▶ Leyendo coordinación base: {DATA_COORD_FILE}")
with open(DATA_COORD_FILE) as f:
    base_json = json.load(f)

processed_base = {}
for esc in base_json:
    sid = esc.get("scenario_id")
    if sid:
        tmp = esc.copy()
        tmp.pop("_id", None); tmp.pop("scenario_id", None)
        eliminar_timestamp(tmp)
        processed_base[sid] = tmp

# ───────────────────── GENERAR LISTA DE PARES ────────────────────────
relay_pairs, total_pairs = [], 0

for sid, esc_data in processed_base.items():
    if sid not in csv_queues:
        print(f"  ⚠  Sin ajustes en CSV para {sid}; omitido.")
        continue

    for line_key, line_val in esc_data.items():
        if not (isinstance(line_val, dict) and "scenarios" in line_val):
            continue

        for fault_key, fault_val in line_val["scenarios"].items():
            main_orig = fault_val.get("main")
            backups   = fault_val.get("backups", [])

            if not main_orig or not backups:      # seguridad
                continue

            main_base = copy.deepcopy(main_orig)
            main_base["line"] = line_key         # registrar línea

            for bk_orig in backups:
                if not isinstance(bk_orig, dict):
                    continue

                # ─── tomar siguiente renglón del CSV (orden original) ──
                if not csv_queues[sid]:
                    raise ValueError(f"CSV sin suficientes filas para {sid}")

                ajustes = csv_queues[sid].popleft()

                main_info = copy.deepcopy(main_base)
                bk_info   = copy.deepcopy(bk_orig)

                # aplicar ajustes principal / respaldo
                main_info["TDS"]     = ajustes["TDS_principal"]
                main_info["pick_up"] = ajustes["Pickup_princ"]
                bk_info["TDS"]       = ajustes["TDS_respaldo"]
                bk_info["pick_up"]   = ajustes["Pickup_resp"]

                # recalcular Time_out
                main_info["Time_out"] = calc_timeout(
                    main_info.get("Ishc"), main_info.get("pick_up"), main_info.get("TDS")
                )
                bk_info["Time_out"] = calc_timeout(
                    bk_info.get("Ishc"), bk_info.get("pick_up"), bk_info.get("TDS")
                )

                relay_pairs.append({
                    "scenario_id": sid,
                    "fault": fault_key,
                    "main_relay": main_info,
                    "backup_relay": bk_info,
                })
                total_pairs += 1

print(f"▶ Pares generados: {total_pairs}")

# ───────────────────── GUARDAR RESULTADO ─────────────────────────────
os.makedirs(os.path.dirname(OUTPUT_PAIRS_FILE), exist_ok=True)
with open(OUTPUT_PAIRS_FILE, "w") as f:
    json.dump(relay_pairs, f, indent=2)

print(f"✔ JSON guardado en: {OUTPUT_PAIRS_FILE}")


▶ Leyendo ajustes de: /Users/gustavo/Documents/Projects/TESIS_UNAL/ADAPTIVE_ALGORITHM/data/processed/model/results/TDS_Pickup_todos_escenarios.csv
   Escenarios encontrados en CSV: ['scenario_1', 'scenario_2', 'scenario_3', 'scenario_4', 'scenario_5', 'scenario_6', 'scenario_7', 'scenario_8', 'scenario_9', 'scenario_10', 'scenario_11', 'scenario_12', 'scenario_13', 'scenario_14', 'scenario_15', 'scenario_16', 'scenario_17', 'scenario_18', 'scenario_19', 'scenario_20', 'scenario_21', 'scenario_22', 'scenario_23', 'scenario_24', 'scenario_25', 'scenario_26', 'scenario_27', 'scenario_28', 'scenario_29', 'scenario_30', 'scenario_31', 'scenario_32', 'scenario_33', 'scenario_34']
▶ Leyendo coordinación base: /Users/gustavo/Documents/Projects/TESIS_UNAL/ADAPTIVE_ALGORITHM/data/raw/data_coordination.json
  ⚠  Sin ajustes en CSV para scenario_35; omitido.
  ⚠  Sin ajustes en CSV para scenario_36; omitido.
  ⚠  Sin ajustes en CSV para scenario_37; omitido.
  ⚠  Sin ajustes en CSV para scenario_3

In [ ]:
import json
import numpy as np
import os
import copy # Para crear copias independientes de los diccionarios de relés

# --- Constantes para el cálculo de Time_out ---
K = 0.14
N = 0.02
DECIMAL_PLACES = 4

# --- Rutas de los archivos ---
data_coordination_file = "/Users/gustavo/Documents/Projects/TESIS_UNAL/ADAPTIVE_ALGORITHM/data/raw/data_coordination.json"
# datos de los reusltados de la red neuronal
relay_values_file = "/Users/gustavo/Documents/Projects/TESIS_UNAL/ADAPTIVE_ALGORITHM/data/processed/model/results/TDS_Pickup_todos_escenarios.csv"
# El archivo de salida contendrá la LISTA de pares con la nueva estructura
output_pairs_file = "/Users/gustavo/Documents/Projects/TESIS_UNAL/ADAPTIVE_ALGORITHM/data/processed/independent_relay_pairs_transformer.json" # Nuevo nombre para evitar sobreescribir

# --- Función para eliminar timestamps recursivamente ---
def eliminar_timestamp(obj):
    if isinstance(obj, dict):
        obj.pop("timestamp", None)
        for valor in obj.values():
            eliminar_timestamp(valor)
    elif isinstance(obj, list):
        for elemento in obj:
            eliminar_timestamp(elemento)

# --- Función para calcular el tiempo de operación (Time_out) ---
def calculate_operation_time(I_shc, I_pi, TDS):
    # (Sin cambios en la función de cálculo)
    if not all(isinstance(x, (int, float)) for x in [I_shc, I_pi, TDS]): return 0.0
    if any(x is None or x <= 0 for x in [I_pi, TDS]): return 0.0
    if I_shc < 0: return 0.0
    if I_shc == 0: return 0.0
    try:
        if abs(I_pi) < 1e-9: return 0.0
        M = I_shc / I_pi
    except ZeroDivisionError: return 0.0
    if M <= 1: return 0.0
    try:
        denominator = M**N - 1
        if abs(denominator) < 1e-9: return 0.0
        timeout = (K / denominator) * TDS
        return round(timeout, DECIMAL_PLACES) if np.isfinite(timeout) else 0.0
    except (OverflowError, ValueError): return 0.0
    except Exception: return 0.0


# --- Procesamiento Principal ---
relay_pairs_list = []
processed_scenarios_base = {}

try:
    # 1. Cargar y pre-procesar los datos base de coordinación
    print(f"Cargando datos base desde: {data_coordination_file}")
    with open(data_coordination_file, 'r') as archivo:
        datos_coordinacion = json.load(archivo)

    for escenario_base in datos_coordinacion:
        scenario_id = escenario_base.get("scenario_id")
        if scenario_id:
            escenario_procesado = escenario_base.copy()
            escenario_procesado.pop("_id", None)
            escenario_procesado.pop("scenario_id", None)
            eliminar_timestamp(escenario_procesado)
            processed_scenarios_base[scenario_id] = escenario_procesado
    print(f"Datos base cargados y pre-procesados para {len(processed_scenarios_base)} escenarios.")

    # 2. Cargar los valores de los relés (antes 'optimizados')
    print(f"Cargando valores de relés desde: {relay_values_file}")
    with open(relay_values_file, 'r') as archivo:
        # Renombrado: datos_relay_values
        datos_relay_values = json.load(archivo)
    # Renombrado: relay_values_map
    # ASUNCIÓN: La clave DENTRO del JSON ahora es 'relay_values'.
    # Si sigue siendo 'optimized_relay_values', cambia el .get() abajo.
    relay_values_map = {esc.get("scenario_id"): esc.get("relay_values", {})
                        for esc in datos_relay_values if esc.get("scenario_id")}
    print(f"Valores de relés cargados para {len(relay_values_map)} escenarios.")

    # 3. Fusionar, calcular timeouts y CONSTRUIR LISTA DE PARES (con línea en main_relay)
    print("Procesando escenarios para crear pares de relés independientes...")
    pair_count = 0
    for scenario_id, escenario_data in processed_scenarios_base.items():
        print(f"Procesando escenario: {scenario_id}")
        # Renombrado: current_relay_values
        current_relay_values = relay_values_map.get(scenario_id)

        if not current_relay_values:
            print(f"  Advertencia: No se encontraron valores de relé para {scenario_id}. Omitiendo pares de este escenario.")
            continue

        # Iterar sobre las líneas (como "L1-2")
        for linea_key, linea_data in escenario_data.items():
             if isinstance(linea_data, dict) and 'scenarios' in linea_data:
                # Iterar sobre los escenarios internos/fallas (como "90")
                for fault_key, internal_scenario_data in linea_data.get('scenarios', {}).items():
                    main_relay_info_orig = internal_scenario_data.get('main')
                    backups_orig = internal_scenario_data.get('backups', [])

                    if isinstance(main_relay_info_orig, dict) and isinstance(backups_orig, list):
                        # Procesar relé principal UNA VEZ
                        main_relay_info = copy.deepcopy(main_relay_info_orig)
                        main_relay_name = main_relay_info.get('relay')
                        main_time_out = 0.0

                        # --- Añadir la línea principal AL OBJETO main_relay ---
                        main_relay_info['line'] = linea_key
                        # ----------------------------------------------------

                        # Renombrado: current_relay_values
                        if main_relay_name and main_relay_name in current_relay_values:
                            # Renombrado: relay_setting
                            relay_setting = current_relay_values[main_relay_name]
                            main_relay_info['TDS'] = relay_setting.get('TDS')
                            main_relay_info['pick_up'] = relay_setting.get('pickup')
                            ishc = main_relay_info.get('Ishc')
                            pickup = main_relay_info.get('pick_up')
                            tds = main_relay_info.get('TDS')
                            main_time_out = calculate_operation_time(ishc, pickup, tds)
                            main_relay_info['Time_out'] = main_time_out
                        else:
                             ishc = main_relay_info.get('Ishc')
                             pickup = main_relay_info.get('pick_up')
                             tds = main_relay_info.get('TDS')
                             main_time_out = calculate_operation_time(ishc, pickup, tds)
                             main_relay_info['Time_out'] = main_time_out

                        # Iterar sobre CADA relé de respaldo
                        for backup_relay_info_orig in backups_orig:
                            if isinstance(backup_relay_info_orig, dict):
                                backup_relay_info = copy.deepcopy(backup_relay_info_orig)
                                backup_relay_name = backup_relay_info.get('relay')
                                backup_time_out = 0.0

                                # Renombrado: current_relay_values
                                if backup_relay_name and backup_relay_name in current_relay_values:
                                    # Renombrado: relay_setting
                                    relay_setting = current_relay_values[backup_relay_name]
                                    backup_relay_info['TDS'] = relay_setting.get('TDS')
                                    backup_relay_info['pick_up'] = relay_setting.get('pickup')
                                    ishc = backup_relay_info.get('Ishc')
                                    pickup = backup_relay_info.get('pick_up')
                                    tds = backup_relay_info.get('TDS')
                                    backup_time_out = calculate_operation_time(ishc, pickup, tds)
                                    backup_relay_info['Time_out'] = backup_time_out
                                else:
                                    ishc = backup_relay_info.get('Ishc')
                                    pickup = backup_relay_info.get('pick_up')
                                    tds = backup_relay_info.get('TDS')
                                    backup_time_out = calculate_operation_time(ishc, pickup, tds)
                                    backup_relay_info['Time_out'] = backup_time_out

                                # Crear el diccionario del par (SIN 'line' de nivel superior)
                                pair_entry = {
                                    "scenario_id": scenario_id,
                                    # "line": linea_key, # <-- Eliminado de aquí
                                    "fault": fault_key,
                                    "main_relay": main_relay_info, # Ya contiene 'line'
                                    "backup_relay": backup_relay_info # Mantiene su 'line' interna opcional
                                }
                                relay_pairs_list.append(pair_entry)
                                pair_count += 1

    print(f"Procesamiento completado. Se generaron {pair_count} pares de relés.")

    # 4. Guardar la LISTA de pares en el archivo de salida
    print(f"Guardando lista de pares en: {output_pairs_file}")
    output_dir = os.path.dirname(output_pairs_file)
    if output_dir: os.makedirs(output_dir, exist_ok=True)

    with open(output_pairs_file, 'w') as f:
        json.dump(relay_pairs_list, f, indent=2)

    print("¡Proceso completado exitosamente!")

except FileNotFoundError as e:
    print(f"Error CRÍTICO: No se pudo encontrar el archivo: {e.filename}")
    print("Verifica que las rutas y nombres de archivo sean correctos (especialmente 'relay_values.json').")
except json.JSONDecodeError as e:
    print(f"Error CRÍTICO: El archivo JSON está mal formado: {e}")
except KeyError as e:
    print(f"Error: Falta una clave esperada en los datos: {e}")
except Exception as e:
    import traceback
    print(f"Error inesperado durante el procesamiento: {e}")
    traceback.print_exc()

Cargando datos base desde: /Users/gustavo/Documents/Projects/TESIS_UNAL/ADAPTIVE_ALGORITHM/data/raw/data_coordination.json
Datos base cargados y pre-procesados para 68 escenarios.
Cargando valores de relés desde: /Users/gustavo/Documents/Projects/TESIS_UNAL/ADAPTIVE_ALGORITHM/data/raw/optimized_relay_values.json
Valores de relés cargados para 68 escenarios.
Procesando escenarios para crear pares de relés independientes...
Procesando escenario: scenario_1
Procesando escenario: scenario_2
Procesando escenario: scenario_3
Procesando escenario: scenario_4
Procesando escenario: scenario_5
Procesando escenario: scenario_6
Procesando escenario: scenario_7
Procesando escenario: scenario_8
Procesando escenario: scenario_9
Procesando escenario: scenario_10
Procesando escenario: scenario_11
Procesando escenario: scenario_12
Procesando escenario: scenario_13
Procesando escenario: scenario_14
Procesando escenario: scenario_15
Procesando escenario: scenario_16
Procesando escenario: scenario_17
Proce